In [1]:
import pymongo
from pymongo import MongoClient
import pyspark
from pyspark.sql.types import  *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
client = MongoClient("172.23.149.210", 27017)
db = client['cardano_silver']
db2 = client['cardano_bronze']

In [3]:
config = pyspark.SparkConf().setAll([
    ('spark.executor.memory', '90g'), 
    ('spark.executor.cores', '5'), # number of cores to use on each executor
    ('spark.cores.max', '5'), # the maximum amount of CPU cores to request for the application from across the cluster
    ('spark.driver.memory','20g'),
    ('spark.executor.instances', '1'),
    ('spark.worker.cleanup.enabled', 'true'),
    ('spark.worker.cleanup.interval', '60'),
    ('spark.worker.cleanup.appDataTtl', '60'),
    ('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:10.0.2'),
    ('spark.mongodb.output.writeConcern.wTimeoutMS','1000000'),
    ('spark.mongodb.output.writeConcern.socketTimeoutMS','1000000'),
    ('spark.mongodb.output.writeConcern.connectTimeoutMS','1000000'),
    ("neo4j.url", "bolt://172.23.149.210:7687"),
    ("neo4j.authentication.type", "basic"),
    ("neo4j.authentication.basic.username", "neo4j"),
    ("neo4j.authentication.basic.password", "cardano")
])

In [4]:
spark = SparkSession \
    .builder \
	.config(conf=config) \
    .appName("Neo4j-Stream") \
    .master("spark://172.23.149.210:7077") \
    .getOrCreate()

23/01/13 22:41:40 WARN Utils: Your hostname, cardano-druid resolves to a loopback address: 127.0.0.1; using 172.23.149.210 instead (on interface ens3)
23/01/13 22:41:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.mongodb.spark#mongo-spark-connector added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e2b45ca8-2254-480f-a052-b8e8656d26e2;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector;10.0.2 in central
	found org.mongodb#mongodb-driver-sync;4.5.1 in central
	[4.5.1] org.mongodb#mongodb-driver-sync;[4.5.0,4.5.99)
	found org.mongodb#bson;4.5.1 in central
	found org.mongodb#mongodb-driver-core;4.5.1 in central
:: resolution report :: resolve 2983ms :: artifacts dl 21ms
	:: modules in use:
	org.mongodb#bson;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-core;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-sync;4.5.1 from central in [default]
	org.mongodb.spark#mongo-spark-connector;10.0.2 from central in [default]
	---------------------------------------------------------------------
	|                  | 

In [5]:
schema = StructType([ \
    StructField("address", StringType(), True) \
])

In [5]:
schema2 = StructType([ \
    StructField("input_addr", StringType(), True), \
    StructField("output_addr", StringType(), True), \
    StructField("tx_hash", StringType(), True), \
    StructField("input_ADA_value", IntegerType(), True), \
    StructField("output_ADA_value", IntegerType(), True), \
])

In [6]:
schema3 = StructType([ \
    StructField("address", StringType(), True), \
    StructField("id", IntegerType(), True) \
])

In [5]:
schema4 = StructType([ \
    StructField("input_addr", StringType(), True), \
    StructField("output_addr", StringType(), True), \
    StructField("tx_hash", StringType(), True), \
    StructField("tx_id", IntegerType(), True), \
    StructField("input_ADA_value", IntegerType(), True), \
    StructField("output_ADA_value", IntegerType(), True), \
])

In [8]:
schema5 = StructType([ \
    StructField("hash", StringType(), True), \
    StructField("id", IntegerType(), True) \
])

In [9]:
schema6 = StructType([ \
    StructField("input_addr", StringType(), True), \
    StructField("output_addr", StringType(), True), \
    StructField("tx_hash", StringType(), True), \
    StructField("tx_id", IntegerType(), True), \
    StructField("input_ADA_value", IntegerType(), True), \
    StructField("output_ADA_value", IntegerType(), True), \
    StructField("input_id", IntegerType(), True), \
    StructField("output_id", IntegerType(), True), \
])

In [6]:
tx_out = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_bronze') \
  	.option('spark.mongodb.collection', 'node.public.tx_out') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema) \
  	.load()

In [7]:
tx_out.createOrReplaceTempView("tx_out")

In [8]:
addresses_raw = spark.sql("SELECT DISTINCT address FROM tx_out")

In [9]:
addresses_raw.write.format("mongodb") \
   .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .mode("append") \
   .option('spark.mongodb.database', 'cardano_bronze') \
   .option('spark.mongodb.collection', 'addresses') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .save()

22/12/24 12:54:20 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.
22/12/24 12:54:36 WARN TaskSetManager: Lost task 21.0 in stage 0.0 (TID 21) (172.23.149.210 executor 0): com.mongodb.MongoSocketReadException: Prematurely reached end of stream
	at com.mongodb.internal.connection.SocketStream.read(SocketStream.java:112)
	at com.mongodb.internal.connection.SocketStream.read(SocketStream.java:131)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveResponseBuffers(InternalStreamConnection.java:718)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveMessageWithAdditionalTimeout(InternalStreamConnection.java:576)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveCommandMessageResponse(InternalStreamConnection.java:415)
	at com.mongodb.internal.connection.InternalStreamConnection.sendAndReceive(InternalStreamConnection.java:342)
	at com.mongodb.internal.connection.Usage

22/12/24 12:56:06 WARN TaskSetManager: Lost task 478.0 in stage 0.0 (TID 483) (172.23.149.210 executor 0): com.mongodb.MongoSocketReadException: Prematurely reached end of stream
	at com.mongodb.internal.connection.SocketStream.read(SocketStream.java:112)
	at com.mongodb.internal.connection.SocketStream.read(SocketStream.java:131)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveResponseBuffers(InternalStreamConnection.java:718)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveMessageWithAdditionalTimeout(InternalStreamConnection.java:576)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveCommandMessageResponse(InternalStreamConnection.java:415)
	at com.mongodb.internal.connection.InternalStreamConnection.sendAndReceive(InternalStreamConnection.java:342)
	at com.mongodb.internal.connection.UsageTrackingInternalConnection.sendAndReceive(UsageTrackingInternalConnection.java:116)
	at com.mongodb.internal.connection.DefaultConnectionP

In [11]:
addresses_raw.write.format("org.neo4j.spark.DataSource") \
  .mode("append") \
  .option("labels", "Address") \
  .option("node.keys", "address") \
  .save()

In [ ]:
# Create id for each address in neo4j using internal ids using:
# CALL apoc.periodic.iterate("MATCH (n:Address) RETURN n","SET n.id = id(n)",{batchSize:10000, parallel:false})

In [10]:
addresses_id = spark.read.format("org.neo4j.spark.DataSource") \
  .option("labels", "Address") \
  .load()

In [11]:
addresses_id.createOrReplaceTempView("addresses_id")

In [12]:
addresses_id = spark.sql("SELECT id, address FROM addresses_id")

In [13]:
addresses_id.write.format("mongodb") \
   .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .mode("append") \
   .option('spark.mongodb.database', 'cardano_silver') \
   .option('spark.mongodb.collection', 'addresses') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .save()

22/12/24 13:13:56 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.


### Transaction Network Processing

In [10]:
# read only the tx id and hash
tx = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_bronze') \
  	.option('spark.mongodb.collection', 'node.public.tx') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema5) \
  	.load()

In [11]:
# store the tx id and hash
tx.write.format("mongodb") \
   .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .mode("append") \
   .option('spark.mongodb.database', 'cardano_bronze') \
   .option('spark.mongodb.collection', 'tx_ids') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .save()

23/01/07 14:30:34 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.


In [12]:
# read the stored collection
tx = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_bronze') \
  	.option('spark.mongodb.collection', 'tx_ids') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema5) \
  	.load()

In [13]:
tx.createOrReplaceTempView("tx")

In [ ]:
TxNetwork = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_silver') \
  	.option('spark.mongodb.collection', 'transaction_network') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema2) \
  	.load()

In [ ]:
TxNetwork.createOrReplaceTempView("TxNetwork")

In [14]:
# join the transaction network to tx in order to add tx_id to the transaction network collection
transactions_id = spark.sql("SELECT tx_hash, id as tx_id, input_addr, output_addr, input_ADA_value, output_ADA_value FROM TxNetwork LEFT JOIN tx on TxNetwork.tx_hash = tx.hash")

In [15]:
transactions_id.write.format("mongodb") \
   .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .mode("append") \
   .option('spark.mongodb.database', 'cardano_bronze') \
   .option('spark.mongodb.collection', 'neo4j_stream_stage1') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .save()

23/01/07 20:53:42 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.
23/01/07 21:20:16 WARN TaskSetManager: Lost task 2241.0 in stage 0.0 (TID 2241) (172.23.149.210 executor 0): com.mongodb.MongoSocketReadException: Prematurely reached end of stream
	at com.mongodb.internal.connection.SocketStream.read(SocketStream.java:112)
	at com.mongodb.internal.connection.SocketStream.read(SocketStream.java:131)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveResponseBuffers(InternalStreamConnection.java:718)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveMessageWithAdditionalTimeout(InternalStreamConnection.java:576)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveCommandMessageResponse(InternalStreamConnection.java:415)
	at com.mongodb.internal.connection.InternalStreamConnection.sendAndReceive(InternalStreamConnection.java:342)
	at com.mongodb.internal.connection.U

23/01/07 21:23:57 WARN TaskSetManager: Lost task 74.0 in stage 1.0 (TID 2798) (172.23.149.210 executor 0): com.mongodb.MongoConnectionPoolClearedException: Connection pool for ServerId{clusterId=ClusterId{value='63b9d159acbfb6168c7e4a6b', description='null'}, address=172.23.149.210:27017} is paused because another operation failed
	at com.mongodb.internal.connection.DefaultConnectionPool$StateAndGeneration.throwIfClosedOrPaused(DefaultConnectionPool.java:1533)
	at com.mongodb.internal.connection.DefaultConnectionPool.get(DefaultConnectionPool.java:174)
	at com.mongodb.internal.connection.DefaultConnectionPool.get(DefaultConnectionPool.java:166)
	at com.mongodb.internal.connection.DefaultServer.getConnection(DefaultServer.java:103)
	at com.mongodb.internal.binding.ClusterBinding$ClusterBindingConnectionSource.getConnection(ClusterBinding.java:175)
	at com.mongodb.client.internal.ClientSessionBinding$SessionBindingConnectionSource.getConnection(ClientSessionBinding.java:192)
	at com.mong

23/01/07 22:30:13 WARN TaskSetManager: Lost task 91.0 in stage 4.0 (TID 2916) (172.23.149.210 executor 0): com.mongodb.MongoSocketReadException: Prematurely reached end of stream
	at com.mongodb.internal.connection.SocketStream.read(SocketStream.java:112)
	at com.mongodb.internal.connection.SocketStream.read(SocketStream.java:131)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveResponseBuffers(InternalStreamConnection.java:718)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveMessageWithAdditionalTimeout(InternalStreamConnection.java:576)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveCommandMessageResponse(InternalStreamConnection.java:415)
	at com.mongodb.internal.connection.InternalStreamConnection.sendAndReceive(InternalStreamConnection.java:342)
	at com.mongodb.internal.connection.UsageTrackingInternalConnection.sendAndReceive(UsageTrackingInternalConnection.java:116)
	at com.mongodb.internal.connection.DefaultConnectionP

23/01/07 23:29:45 WARN TaskSetManager: Lost task 174.0 in stage 4.0 (TID 3004) (172.23.149.210 executor 0): com.mongodb.MongoSocketReadException: Exception receiving message
	at com.mongodb.internal.connection.InternalStreamConnection.translateReadException(InternalStreamConnection.java:707)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveMessageWithAdditionalTimeout(InternalStreamConnection.java:579)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveCommandMessageResponse(InternalStreamConnection.java:415)
	at com.mongodb.internal.connection.InternalStreamConnection.sendAndReceive(InternalStreamConnection.java:342)
	at com.mongodb.internal.connection.UsageTrackingInternalConnection.sendAndReceive(UsageTrackingInternalConnection.java:116)
	at com.mongodb.internal.connection.DefaultConnectionPool$PooledConnection.sendAndReceive(DefaultConnectionPool.java:643)
	at com.mongodb.internal.connection.CommandProtocolImpl.execute(CommandProtocolImpl.java:71

In [5]:
transactions_id = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_bronze') \
  	.option('spark.mongodb.collection', 'neo4j_stream_stage1') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema4) \
  	.load()

In [6]:
# divide the transactions to avoid memory or storage related issues
transactions_id_1 = transactions_id.filter(col("tx_id").between(0,12000000))

In [7]:
transactions_id_1.write.format("mongodb") \
   .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .mode("append") \
   .option('spark.mongodb.database', 'cardano_bronze') \
   .option('spark.mongodb.collection', 'neo4j_stream_stage2_1') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .save()

23/01/10 15:38:34 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.


In [6]:
addresses = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_silver') \
  	.option('spark.mongodb.collection', 'addresses') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema3) \
  	.load()

In [7]:
addresses.createOrReplaceTempView("addresses")

In [8]:
transactions_id_1 = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_bronze') \
  	.option('spark.mongodb.collection', 'neo4j_stream_stage2_1') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema4) \
  	.load()

In [9]:
transactions_id_1.createOrReplaceTempView("transactions_id_1")

In [10]:
# join transaction network to addresses to get the input address ids
input_addresses = spark.sql("SELECT transactions_id_1.tx_hash, transactions_id_1.tx_id, transactions_id_1.input_addr, transactions_id_1.output_addr, transactions_id_1.input_ADA_value, transactions_id_1.output_ADA_value, addresses.id as input_id FROM transactions_id_1 LEFT JOIN addresses on transactions_id_1.input_addr = addresses.address")

In [11]:
input_addresses.createOrReplaceTempView("input_addresses")

In [12]:
# join the previous dataframe to addresses to get the output ids
TxNet = spark.sql("SELECT input_addresses.tx_hash, input_addresses.tx_id, input_addresses.input_addr, input_addresses.output_addr, input_addresses.input_ADA_value, input_addresses.output_ADA_value, input_addresses.input_id, addresses.id as output_id FROM input_addresses LEFT JOIN addresses on input_addresses.output_addr = addresses.address")

In [13]:
# store the result, which includes the original transaction network with columns for tx_id, input addr. id, 
# and output addr. id to make neo4j queries more effecient
TxNet.write.format("mongodb") \
   .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .mode("append") \
   .option('spark.mongodb.database', 'cardano_bronze') \
   .option('spark.mongodb.collection', 'neo4j_stream_stage3_1') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .save()

23/01/10 16:50:41 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.
23/01/10 17:09:43 WARN TaskSetManager: Lost task 542.0 in stage 0.0 (TID 542) (172.23.149.210 executor 0): com.mongodb.MongoSocketReadException: Exception receiving message
	at com.mongodb.internal.connection.InternalStreamConnection.translateReadException(InternalStreamConnection.java:707)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveMessageWithAdditionalTimeout(InternalStreamConnection.java:579)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveCommandMessageResponse(InternalStreamConnection.java:415)
	at com.mongodb.internal.connection.InternalStreamConnection.sendAndReceive(InternalStreamConnection.java:342)
	at com.mongodb.internal.connection.UsageTrackingInternalConnection.sendAndReceive(UsageTrackingInternalConnection.java:116)
	at com.mongodb.internal.connection.DefaultConnectionPool$PooledConnection

In [6]:
# repeat the previous process with a new part of the original data
transactions_id = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_bronze') \
  	.option('spark.mongodb.collection', 'neo4j_stream_stage1') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema4) \
  	.load()

In [7]:
transactions_id_2 = transactions_id.filter(col("tx_id").between(12000001,28000000))

In [8]:
transactions_id_2.write.format("mongodb") \
   .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .mode("append") \
   .option('spark.mongodb.database', 'cardano_bronze') \
   .option('spark.mongodb.collection', 'neo4j_stream_stage2_2') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .save()

23/01/10 17:54:21 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.


In [6]:
addresses = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_silver') \
  	.option('spark.mongodb.collection', 'addresses') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema3) \
  	.load()

In [7]:
addresses.createOrReplaceTempView("addresses")

In [8]:
transactions_id_2 = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_bronze') \
  	.option('spark.mongodb.collection', 'neo4j_stream_stage2_2') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema4) \
  	.load()

In [9]:
transactions_id_2.createOrReplaceTempView("transactions_id_2")

In [10]:
input_addresses = spark.sql("SELECT transactions_id_2.tx_hash, transactions_id_2.tx_id, transactions_id_2.input_addr, transactions_id_2.output_addr, transactions_id_2.input_ADA_value, transactions_id_2.output_ADA_value, addresses.id as input_id FROM transactions_id_2 LEFT JOIN addresses on transactions_id_2.input_addr = addresses.address")

In [11]:
input_addresses.createOrReplaceTempView("input_addresses")

In [12]:
TxNet = spark.sql("SELECT input_addresses.tx_hash, input_addresses.tx_id, input_addresses.input_addr, input_addresses.output_addr, input_addresses.input_ADA_value, input_addresses.output_ADA_value, input_addresses.input_id, addresses.id as output_id FROM input_addresses LEFT JOIN addresses on input_addresses.output_addr = addresses.address")

In [13]:
TxNet.write.format("mongodb") \
   .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .mode("append") \
   .option('spark.mongodb.database', 'cardano_bronze') \
   .option('spark.mongodb.collection', 'neo4j_stream_stage3_2') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .save()

23/01/10 19:08:19 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.
23/01/10 19:30:23 WARN TaskSetManager: Lost task 88.0 in stage 1.0 (TID 655) (172.23.149.210 executor 0): com.mongodb.MongoSocketReadException: Exception receiving message
	at com.mongodb.internal.connection.InternalStreamConnection.translateReadException(InternalStreamConnection.java:707)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveMessageWithAdditionalTimeout(InternalStreamConnection.java:579)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveCommandMessageResponse(InternalStreamConnection.java:415)
	at com.mongodb.internal.connection.InternalStreamConnection.sendAndReceive(InternalStreamConnection.java:342)
	at com.mongodb.internal.connection.UsageTrackingInternalConnection.sendAndReceive(UsageTrackingInternalConnection.java:116)
	at com.mongodb.internal.connection.DefaultConnectionPool$PooledConnection.

23/01/10 19:47:47 WARN TaskSetManager: Lost task 40.0 in stage 10.0 (TID 1001) (172.23.149.210 executor 0): com.mongodb.MongoSocketReadException: Prematurely reached end of stream
	at com.mongodb.internal.connection.SocketStream.read(SocketStream.java:112)
	at com.mongodb.internal.connection.SocketStream.read(SocketStream.java:131)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveResponseBuffers(InternalStreamConnection.java:718)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveMessageWithAdditionalTimeout(InternalStreamConnection.java:576)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveCommandMessageResponse(InternalStreamConnection.java:415)
	at com.mongodb.internal.connection.InternalStreamConnection.sendAndReceive(InternalStreamConnection.java:342)
	at com.mongodb.internal.connection.UsageTrackingInternalConnection.sendAndReceive(UsageTrackingInternalConnection.java:116)
	at com.mongodb.internal.connection.DefaultConnection

In [5]:
transactions_id = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_bronze') \
  	.option('spark.mongodb.collection', 'neo4j_stream_stage1') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema4) \
  	.load()

In [6]:
transactions_id_3 = transactions_id.filter(col("tx_id").between(28000001,34000000))

In [7]:
transactions_id_3.write.format("mongodb") \
   .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .mode("append") \
   .option('spark.mongodb.database', 'cardano_bronze') \
   .option('spark.mongodb.collection', 'neo4j_stream_stage2_3') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .save()

23/01/10 20:36:54 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.


In [6]:
addresses = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_silver') \
  	.option('spark.mongodb.collection', 'addresses') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema3) \
  	.load()

In [7]:
addresses.createOrReplaceTempView("addresses")

In [8]:
transactions_id_3 = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_bronze') \
  	.option('spark.mongodb.collection', 'neo4j_stream_stage2_3') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema4) \
  	.load()

In [9]:
transactions_id_3.createOrReplaceTempView("transactions_id_3")

In [10]:
input_addresses = spark.sql("SELECT transactions_id_3.tx_hash, transactions_id_3.tx_id, transactions_id_3.input_addr, transactions_id_3.output_addr, transactions_id_3.input_ADA_value, transactions_id_3.output_ADA_value, addresses.id as input_id FROM transactions_id_3 LEFT JOIN addresses on transactions_id_3.input_addr = addresses.address")

In [11]:
input_addresses.createOrReplaceTempView("input_addresses")

In [12]:
TxNet = spark.sql("SELECT input_addresses.tx_hash, input_addresses.tx_id, input_addresses.input_addr, input_addresses.output_addr, input_addresses.input_ADA_value, input_addresses.output_ADA_value, input_addresses.input_id, addresses.id as output_id FROM input_addresses LEFT JOIN addresses on input_addresses.output_addr = addresses.address")

In [13]:
TxNet.write.format("mongodb") \
   .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .mode("append") \
   .option('spark.mongodb.database', 'cardano_bronze') \
   .option('spark.mongodb.collection', 'neo4j_stream_stage3_3') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .save()

23/01/10 22:18:05 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.


In [5]:
transactions_id = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_bronze') \
  	.option('spark.mongodb.collection', 'neo4j_stream_stage1') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema4) \
  	.load()

In [6]:
transactions_id_4 = transactions_id.filter(col("tx_id").between(34000001,40000000))

In [7]:
transactions_id_4.write.format("mongodb") \
   .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .mode("append") \
   .option('spark.mongodb.database', 'cardano_bronze') \
   .option('spark.mongodb.collection', 'neo4j_stream_stage2_4') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .save()

23/01/11 00:46:21 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.


In [6]:
addresses = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_silver') \
  	.option('spark.mongodb.collection', 'addresses') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema3) \
  	.load()

In [7]:
addresses.createOrReplaceTempView("addresses")

In [8]:
transactions_id_4 = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_bronze') \
  	.option('spark.mongodb.collection', 'neo4j_stream_stage2_4') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema4) \
  	.load()

In [9]:
transactions_id_4.createOrReplaceTempView("transactions_id_4")

In [10]:
input_addresses = spark.sql("SELECT transactions_id_4.tx_hash, transactions_id_4.tx_id, transactions_id_4.input_addr, transactions_id_4.output_addr, transactions_id_4.input_ADA_value, transactions_id_4.output_ADA_value, addresses.id as input_id FROM transactions_id_4 LEFT JOIN addresses on transactions_id_4.input_addr = addresses.address")

In [11]:
input_addresses.createOrReplaceTempView("input_addresses")

In [12]:
TxNet = spark.sql("SELECT input_addresses.tx_hash, input_addresses.tx_id, input_addresses.input_addr, input_addresses.output_addr, input_addresses.input_ADA_value, input_addresses.output_ADA_value, input_addresses.input_id, addresses.id as output_id FROM input_addresses LEFT JOIN addresses on input_addresses.output_addr = addresses.address")

In [13]:
TxNet.write.format("mongodb") \
   .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .mode("append") \
   .option('spark.mongodb.database', 'cardano_bronze') \
   .option('spark.mongodb.collection', 'neo4j_stream_stage3_4') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .save()

23/01/11 02:03:45 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.
23/01/11 02:32:01 WARN TaskSetManager: Lost task 38.0 in stage 2.0 (TID 567) (172.23.149.210 executor 0): com.mongodb.MongoSocketReadException: Exception receiving message
	at com.mongodb.internal.connection.InternalStreamConnection.translateReadException(InternalStreamConnection.java:707)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveMessageWithAdditionalTimeout(InternalStreamConnection.java:579)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveCommandMessageResponse(InternalStreamConnection.java:415)
	at com.mongodb.internal.connection.InternalStreamConnection.sendAndReceive(InternalStreamConnection.java:342)
	at com.mongodb.internal.connection.UsageTrackingInternalConnection.sendAndReceive(UsageTrackingInternalConnection.java:116)
	at com.mongodb.internal.connection.DefaultConnectionPool$PooledConnection.

23/01/11 03:31:30 WARN TaskSetManager: Lost task 135.0 in stage 10.0 (TID 966) (172.23.149.210 executor 0): com.mongodb.MongoSocketReadException: Prematurely reached end of stream
	at com.mongodb.internal.connection.SocketStream.read(SocketStream.java:112)
	at com.mongodb.internal.connection.SocketStream.read(SocketStream.java:131)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveResponseBuffers(InternalStreamConnection.java:718)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveMessageWithAdditionalTimeout(InternalStreamConnection.java:576)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveCommandMessageResponse(InternalStreamConnection.java:415)
	at com.mongodb.internal.connection.InternalStreamConnection.sendAndReceive(InternalStreamConnection.java:342)
	at com.mongodb.internal.connection.UsageTrackingInternalConnection.sendAndReceive(UsageTrackingInternalConnection.java:116)
	at com.mongodb.internal.connection.DefaultConnection

23/01/11 03:59:39 WARN TaskSetManager: Lost task 186.0 in stage 10.0 (TID 1026) (172.23.149.210 executor 0): com.mongodb.MongoSocketReadException: Prematurely reached end of stream
	at com.mongodb.internal.connection.SocketStream.read(SocketStream.java:112)
	at com.mongodb.internal.connection.SocketStream.read(SocketStream.java:131)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveResponseBuffers(InternalStreamConnection.java:718)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveMessageWithAdditionalTimeout(InternalStreamConnection.java:576)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveCommandMessageResponse(InternalStreamConnection.java:415)
	at com.mongodb.internal.connection.InternalStreamConnection.sendAndReceive(InternalStreamConnection.java:342)
	at com.mongodb.internal.connection.UsageTrackingInternalConnection.sendAndReceive(UsageTrackingInternalConnection.java:116)
	at com.mongodb.internal.connection.DefaultConnectio

In [5]:
transactions_id = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_bronze') \
  	.option('spark.mongodb.collection', 'neo4j_stream_stage1') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema4) \
  	.load()

In [6]:
transactions_id_5 = transactions_id.filter(col("tx_id").between(40000001,58400000))

In [7]:
transactions_id_5.write.format("mongodb") \
   .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .mode("append") \
   .option('spark.mongodb.database', 'cardano_bronze') \
   .option('spark.mongodb.collection', 'neo4j_stream_stage2_5') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .save()

23/01/11 14:11:06 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.


In [6]:
addresses = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_silver') \
  	.option('spark.mongodb.collection', 'addresses') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema3) \
  	.load()

In [7]:
addresses.createOrReplaceTempView("addresses")

In [8]:
transactions_id_5 = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_bronze') \
  	.option('spark.mongodb.collection', 'neo4j_stream_stage2_5') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema4) \
  	.load()

In [9]:
transactions_id_5.createOrReplaceTempView("transactions_id_5")

In [10]:
input_addresses = spark.sql("SELECT transactions_id_5.tx_hash, transactions_id_5.tx_id, transactions_id_5.input_addr, transactions_id_5.output_addr, transactions_id_5.input_ADA_value, transactions_id_5.output_ADA_value, addresses.id as input_id FROM transactions_id_5 LEFT JOIN addresses on transactions_id_5.input_addr = addresses.address")

In [11]:
input_addresses.createOrReplaceTempView("input_addresses")

In [12]:
TxNet = spark.sql("SELECT input_addresses.tx_hash, input_addresses.tx_id, input_addresses.input_addr, input_addresses.output_addr, input_addresses.input_ADA_value, input_addresses.output_ADA_value, input_addresses.input_id, addresses.id as output_id FROM input_addresses LEFT JOIN addresses on input_addresses.output_addr = addresses.address")

In [13]:
TxNet.write.format("mongodb") \
   .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .mode("append") \
   .option('spark.mongodb.database', 'cardano_bronze') \
   .option('spark.mongodb.collection', 'neo4j_stream_stage3_5') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .save()

23/01/11 17:30:00 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.
23/01/11 17:50:16 WARN TaskSetManager: Lost task 351.0 in stage 0.0 (TID 351) (172.23.149.210 executor 0): com.mongodb.MongoSocketReadException: Exception receiving message
	at com.mongodb.internal.connection.InternalStreamConnection.translateReadException(InternalStreamConnection.java:707)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveMessageWithAdditionalTimeout(InternalStreamConnection.java:579)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveCommandMessageResponse(InternalStreamConnection.java:415)
	at com.mongodb.internal.connection.InternalStreamConnection.sendAndReceive(InternalStreamConnection.java:342)
	at com.mongodb.internal.connection.UsageTrackingInternalConnection.sendAndReceive(UsageTrackingInternalConnection.java:116)
	at com.mongodb.internal.connection.DefaultConnectionPool$PooledConnection

## Neo4j Streaming

### Part 1

In [10]:
TxNet = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_bronze') \
  	.option('spark.mongodb.collection', 'neo4j_stream_stage3_1') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema6) \
  	.load()

In [ ]:
TxNet.write.format("org.neo4j.spark.DataSource") \
  .mode("append") \
  .option("url", "bolt://172.23.149.210:7687") \
  .option("query","MATCH (a1:Address {id: event.input_id}) MATCH (a2:Address {id: event.output_id}) CREATE (a1)-[:TRANSACTED_WITH {tx_hash: event.tx_hash, tx_id: event.tx_id, input_ADA_value: event.input_ADA_value, output_ADA_value: event.output_ADA_value}]->(a2)") \
  .save()

In [6]:
TxNet.createOrReplaceTempView("TxNet")

In [7]:
clusters = spark.sql("SELECT t1.tx_id, t1.input_id as input1, t2.input_id as input2 FROM TxNet t1 LEFT JOIN TxNet t2 on t1.tx_id = t2.tx_id WHERE t1.input_id != t2.input_id")

In [8]:
clusters.createOrReplaceTempView("clusters")

In [9]:
clusters_final = spark.sql("SELECT distinct tx_id, input1, input2 FROM clusters")

In [ ]:
clusters_final.write.format("org.neo4j.spark.DataSource") \
  .mode("append") \
  .option("url", "bolt://172.23.149.210:7687") \
  .option("query","MATCH (a1:Address {id: event.input1}) MATCH (a2:Address {id: event.input2}) CREATE (a1)-[:ASSOCIATED_WITH]->(a2)") \
  .save()

23/01/12 18:46:20 WARN TaskSetManager: Lost task 434.0 in stage 0.0 (TID 434) (172.23.149.210 executor 0): com.mongodb.MongoSocketReadException: Prematurely reached end of stream
	at com.mongodb.internal.connection.SocketStream.read(SocketStream.java:112)
	at com.mongodb.internal.connection.SocketStream.read(SocketStream.java:131)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveResponseBuffers(InternalStreamConnection.java:718)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveMessageWithAdditionalTimeout(InternalStreamConnection.java:576)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveCommandMessageResponse(InternalStreamConnection.java:415)
	at com.mongodb.internal.connection.InternalStreamConnection.sendAndReceive(InternalStreamConnection.java:342)
	at com.mongodb.internal.connection.UsageTrackingInternalConnection.sendAndReceive(UsageTrackingInternalConnection.java:116)
	at com.mongodb.internal.connection.DefaultConnectionP

23/01/12 18:50:22 WARN TaskSetManager: Lost task 417.0 in stage 1.0 (TID 1014) (172.23.149.210 executor 0): com.mongodb.MongoSocketReadException: Prematurely reached end of stream
	at com.mongodb.internal.connection.SocketStream.read(SocketStream.java:112)
	at com.mongodb.internal.connection.SocketStream.read(SocketStream.java:131)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveResponseBuffers(InternalStreamConnection.java:718)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveMessageWithAdditionalTimeout(InternalStreamConnection.java:576)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveCommandMessageResponse(InternalStreamConnection.java:415)
	at com.mongodb.internal.connection.InternalStreamConnection.sendAndReceive(InternalStreamConnection.java:342)
	at com.mongodb.internal.connection.UsageTrackingInternalConnection.sendAndReceive(UsageTrackingInternalConnection.java:116)
	at com.mongodb.internal.connection.DefaultConnection

### Part 2

In [ ]:
TxNet = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_bronze') \
  	.option('spark.mongodb.collection', 'neo4j_stream_stage3_2') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema6) \
  	.load()

In [ ]:
TxNet.write.format("org.neo4j.spark.DataSource") \
  .mode("append") \
  .option("url", "bolt://172.23.149.210:7687") \
  .option("query","MATCH (a1:Address {id: event.input_id}) MATCH (a2:Address {id: event.output_id}) CREATE (a1)-[:TRANSACTED_WITH {tx_hash: event.tx_hash, tx_id: event.tx_id, input_ADA_value: event.input_ADA_value, output_ADA_value: event.output_ADA_value}]->(a2)") \
  .save()

In [ ]:
TxNet.createOrReplaceTempView("TxNet")

In [ ]:
clusters = spark.sql("SELECT t1.tx_id, t1.input_id as input1, t2.input_id as input2 FROM TxNet t1 LEFT JOIN TxNet t2 on t1.tx_id = t2.tx_id WHERE t1.input_id != t2.input_id")

In [ ]:
clusters.createOrReplaceTempView("clusters")

In [ ]:
clusters_final = spark.sql("SELECT distinct tx_id, input1, input2 FROM clusters")

In [ ]:
clusters_final.write.format("org.neo4j.spark.DataSource") \
  .mode("append") \
  .option("url", "bolt://172.23.149.210:7687") \
  .option("query","MATCH (a1:Address {id: event.input1}) MATCH (a2:Address {id: event.input2}) CREATE (a1)-[:ASSOCIATED_WITH]->(a2)") \
  .save()

### Part 3

In [ ]:
TxNet = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_bronze') \
  	.option('spark.mongodb.collection', 'neo4j_stream_stage3_3') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema6) \
  	.load()

In [ ]:
TxNet.write.format("org.neo4j.spark.DataSource") \
  .mode("append") \
  .option("url", "bolt://172.23.149.210:7687") \
  .option("query","MATCH (a1:Address {id: event.input_id}) MATCH (a2:Address {id: event.output_id}) CREATE (a1)-[:TRANSACTED_WITH {tx_hash: event.tx_hash, tx_id: event.tx_id, input_ADA_value: event.input_ADA_value, output_ADA_value: event.output_ADA_value}]->(a2)") \
  .save()

In [ ]:
TxNet.createOrReplaceTempView("TxNet")

In [ ]:
clusters = spark.sql("SELECT t1.tx_id, t1.input_id as input1, t2.input_id as input2 FROM TxNet t1 LEFT JOIN TxNet t2 on t1.tx_id = t2.tx_id WHERE t1.input_id != t2.input_id")

In [ ]:
clusters.createOrReplaceTempView("clusters")

In [ ]:
clusters_final = spark.sql("SELECT distinct tx_id, input1, input2 FROM clusters")

In [ ]:
clusters_final.write.format("org.neo4j.spark.DataSource") \
  .mode("append") \
  .option("url", "bolt://172.23.149.210:7687") \
  .option("query","MATCH (a1:Address {id: event.input1}) MATCH (a2:Address {id: event.input2}) CREATE (a1)-[:ASSOCIATED_WITH]->(a2)") \
  .save()

### Part 4

In [ ]:
TxNet = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_bronze') \
  	.option('spark.mongodb.collection', 'neo4j_stream_stage3_4') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema6) \
  	.load()

In [ ]:
TxNet.write.format("org.neo4j.spark.DataSource") \
  .mode("append") \
  .option("url", "bolt://172.23.149.210:7687") \
  .option("query","MATCH (a1:Address {id: event.input_id}) MATCH (a2:Address {id: event.output_id}) CREATE (a1)-[:TRANSACTED_WITH {tx_hash: event.tx_hash, tx_id: event.tx_id, input_ADA_value: event.input_ADA_value, output_ADA_value: event.output_ADA_value}]->(a2)") \
  .save()

In [ ]:
TxNet.createOrReplaceTempView("TxNet")

In [ ]:
clusters = spark.sql("SELECT t1.tx_id, t1.input_id as input1, t2.input_id as input2 FROM TxNet t1 LEFT JOIN TxNet t2 on t1.tx_id = t2.tx_id WHERE t1.input_id != t2.input_id")

In [ ]:
clusters.createOrReplaceTempView("clusters")

In [ ]:
clusters_final = spark.sql("SELECT distinct tx_id, input1, input2 FROM clusters")

In [ ]:
clusters_final.write.format("org.neo4j.spark.DataSource") \
  .mode("append") \
  .option("url", "bolt://172.23.149.210:7687") \
  .option("query","MATCH (a1:Address {id: event.input1}) MATCH (a2:Address {id: event.input2}) CREATE (a1)-[:ASSOCIATED_WITH]->(a2)") \
  .save()

### Part 5

In [ ]:
TxNet = spark.read.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.option('spark.mongodb.database', 'cardano_bronze') \
  	.option('spark.mongodb.collection', 'neo4j_stream_stage3_5') \
	.option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
	.option('spark.mongodb.change.stream.publish.full.document.only','true') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema6) \
  	.load()

In [ ]:
TxNet.write.format("org.neo4j.spark.DataSource") \
  .mode("append") \
  .option("url", "bolt://172.23.149.210:7687") \
  .option("query","MATCH (a1:Address {id: event.input_id}) MATCH (a2:Address {id: event.output_id}) CREATE (a1)-[:TRANSACTED_WITH {tx_hash: event.tx_hash, tx_id: event.tx_id, input_ADA_value: event.input_ADA_value, output_ADA_value: event.output_ADA_value}]->(a2)") \
  .save()

In [ ]:
TxNet.createOrReplaceTempView("TxNet")

In [ ]:
clusters = spark.sql("SELECT t1.tx_id, t1.input_id as input1, t2.input_id as input2 FROM TxNet t1 LEFT JOIN TxNet t2 on t1.tx_id = t2.tx_id WHERE t1.input_id != t2.input_id")

In [ ]:
clusters.createOrReplaceTempView("clusters")

In [ ]:
clusters_final = spark.sql("SELECT distinct tx_id, input1, input2 FROM clusters")

In [ ]:
clusters_final.write.format("org.neo4j.spark.DataSource") \
  .mode("append") \
  .option("url", "bolt://172.23.149.210:7687") \
  .option("query","MATCH (a1:Address {id: event.input1}) MATCH (a2:Address {id: event.input2}) CREATE (a1)-[:ASSOCIATED_WITH]->(a2)") \
  .save()

In [ ]:
last_ind = db2["last_index_neo4j_stream"]
addr_query = {"collection": "neo4j_stream"}
new_count = {"$set":{"last_index": count_records}}
last_ind.update_one(addr_query, new_count)

In [16]:
spark.stop()